In [ ]:
import torch
from PIL import Image
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
from google.colab import drive
import os
import random
import shutil
from torchvision import models

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_PATH = '/content/dataset/raw/'


In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
!mkdir -p /content/dataset/raw

In [ ]:
!cp -r /content/drive/MyDrive/AI_Real_Dataset/raw/train /content/dataset/raw/
!cp -r /content/drive/MyDrive/AI_Real_Dataset/raw/test /content/dataset/raw/
!cp -r /content/drive/MyDrive/AI_Real_Dataset/raw/validation /content/dataset/raw/

In [ ]:
!echo "TRAIN AI:" && ls /content/dataset/raw/train/ai | wc -l
!echo "VAL AI:" && ls /content/dataset/raw/validation/ai | wc -l
!echo "TEST AI:" && ls /content/dataset/raw/test/ai | wc -l

!echo "TRAIN REAL:" && ls /content/dataset/raw/train/real | wc -l
!echo "VAL REAL:" && ls /content/dataset/raw/validation/real | wc -l
!echo "TEST REAL:" && ls /content/dataset/raw/test/real | wc -l

TRAIN AI:
11349
VAL AI:
2431
TEST AI:
2433
TRAIN REAL:
11349
VAL REAL:
2431
TEST REAL:
2433


In [ ]:
train_dataset = datasets.ImageFolder(root='/content/dataset/raw/train', transform=transform_train)
validation_dataset = datasets.ImageFolder(root='/content/dataset/raw/validation', transform=transform_val)
test_dataset = datasets.ImageFolder(root='/content/dataset/raw/test', transform=transform_test)

In [ ]:
test_dataset = datasets.ImageFolder(root='/content/dataset/raw/test', transform=transform_test)
test_dataset.class_to_idx

{'ai': 0, 'real': 1}

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
images, labels = next(iter(train_loader))
print(images.shape)
print(labels)

torch.Size([32, 3, 224, 224])
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 1, 0, 1, 1, 1, 1, 0])


In [ ]:
device=''
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'

print(device)

cuda


In [ ]:
weights = ConvNeXt_Tiny_Weights.IMAGENET1K_V1
model = convnext_tiny(weights=weights)
in_features = model.classifier[2].in_features
model.classifier[2] = nn.Linear(in_features, 1)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth


100%|██████████| 109M/109M [00:00<00:00, 158MB/s] 


In [ ]:
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier[2].parameters():
  param.requires_grad = True

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.classifier[2].parameters(), lr=1e-4)

In [ ]:
scaler = torch.amp.GradScaler()

In [ ]:
def train(model, loader, optimizer, criterion):

    model.train()
    train_loss = 0.0

    for i, (inputs, labels) in enumerate(loader):

        inputs = inputs.to(device)
        labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.amp.autocast(device_type=device):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * inputs.size(0)

    return train_loss / len(loader.dataset)

In [ ]:
def validate(model, loader, criterion):

    model.eval()
    running_loss  = 0.0
    correct = 0
    total = 0

    with torch.no_grad():

        for i, (inputs, labels) in enumerate(loader):

            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

            outputs = model(inputs)
            loss = criterion(outputs, labels)


            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()

            correct += (preds.squeeze(1) == labels.squeeze(1)).sum().item()
            total += labels.size(0)
            running_loss += loss.item() * inputs.size(0)


    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
epochs = 5
PATIENCE = 2
best_val_acc = 0.0
patience_counter = 0

for epoch in range(epochs):

    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, validation_loader, criterion)

    print(f"""Epoch [{epoch+1}/{epochs}]
            Train Loss: {train_loss:.4f}
            Val Loss:   {val_loss:.4f}
            Val Acc:    {val_acc:.4f}
    """)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        print("Early stopping triggered")
        break

Epoch [1/5]
            Train Loss: 0.4154
            Val Loss:   0.2562
            Val Acc:    0.9539
    
Epoch [2/5]
            Train Loss: 0.2078
            Val Loss:   0.1617
            Val Acc:    0.9700
    
Epoch [3/5]
            Train Loss: 0.1431
            Val Loss:   0.1201
            Val Acc:    0.9786
    
Epoch [4/5]
            Train Loss: 0.1113
            Val Loss:   0.0965
            Val Acc:    0.9833
    
Epoch [5/5]
            Train Loss: 0.0914
            Val Loss:   0.0808
            Val Acc:    0.9862
    


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/AI_Real_Dataset/pretrainedRestnet/convnext_tiny_Head.pth")

In [ ]:
for param in model.parameters():
  param.requires_grad = False

for param in model.classifier[2].parameters():
  param.requires_grad = True

for param in model.features[-1].parameters():
    param.requires_grad = True

In [ ]:
optimizer = torch.optim.AdamW(
    [
        {"params": model.features[-1].parameters(), "lr": 1e-5},
        {"params": model.classifier[2].parameters(),     "lr": 1e-4},
    ],
    weight_decay=1e-4
)

criterion = nn.BCEWithLogitsLoss()